# Test Tiny A11y LoRA Model
This notebook loads your LoRA fine-tuned `younglim/tiny-a11y-model` on DeepSeek-Coder and runs prompts.

In [25]:
# Install required packages
!pip install --upgrade pip
!pip install transformers accelerate peft

In [26]:
# Load base model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
lora_model_name = 'younglim/tiny-a11y-model'

tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map='auto')

# Apply LoRA weights
model = PeftModel.from_pretrained(base_model, lora_model_name, device_map='auto')

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/976 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

In [28]:
# Create text-generation pipeline
from transformers import pipeline

# Remove the `device` argument since accelerate already handles it
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Example prompt
prompt = 'Explain why alt text is important for images.'
output = generator(prompt, max_new_tokens=100)
print(output[0]['generated_text'])

Device set to use cuda:0
The model 'PeftModel' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FlexOlmoForCausalLM', '

Explain why alt text is important for images.
       

alt text is important because when an image is displayed on a website, it's important for the person using the image to understand what it's meant to be. The alt text provides the description of the image. This helps users who use screen readers or other assistive technologies to understand the image. When we don't provide an alt text, it might be confusing for users who have difficulty seeing the image.



In [29]:
# Interactive prompt testing
custom_prompt = input('Enter your prompt: ')
result = generator(custom_prompt, max_new_tokens=100)
print(result[0]['generated_text'])

Enter your prompt: What is WCAG 1.4.3
What is WCAG 1.4.33?
	// "WCAG 1.4.33 is a part of the Web Content Accessibility Guidelines (WCAG) 2.0. The main goal of this guideline is to assist visually impaired users by providing a method to understand and navigate the content."

	require_once( 'wp-content/plugins/accessible-media-gallery/includes/class-accessible-media-gallery.php' );

